In [92]:
%pip install Sastrawi

import pandas as pd
import re
import nltk
import math
import sys
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from IPython.display import clear_output

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\Candra\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [93]:
# Load dataset
df_corpus = pd.read_csv('corpus_kemah_jateng_diy - Sheet3.csv')

print(df_corpus.head())

   Doc_ID         Nama_Tempat                 Lokasi  Rating  \
0       1  Kuncen Camp Ground  Kab. Semarang, Jateng       5   
1       2  Kuncen Camp Ground  Kab. Semarang, Jateng       5   
2       3  Kuncen Camp Ground  Kab. Semarang, Jateng       5   
3       4  Kuncen Camp Ground  Kab. Semarang, Jateng       5   
4       5  Kuncen Camp Ground  Kab. Semarang, Jateng       5   

                                         Teks_Mentah  
0  Bagus banget tempatnya, terkonsep dan guide ny...  
1  Sangat menyenangkan untuk camping ceria.\r\nNy...  
2  Tempatnya asri dan sejuk, sudah lumayan ramai ...  
3  3 kali ke sini, sekali ikut acara, 2 kali biki...  
4  Tempat yang cocok untuk acara kemah, kami kema...  


In [94]:
# Import Sastrawi
try:
    from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
except ImportError:
    print("Warning: Sastrawi is not available. Using Dummy Stemmer.")
    class DummyStemmer:
        def stem(self, text):
            return text
    class StemmerFactory:
        def create_stemmer(self):
            return DummyStemmer()

# Safety for NLTK Stopwords      
try:
    from nltk.corpus import stopwords
    stopwords_id = set(stopwords.words('indonesian')) 
except (LookupError, ImportError):
    print("Warning: Indonesian stopwords failed to load. Using minimal manual list.")
    stopwords_id = {"yang", "dan", "di", "ke", "adalah", "dengan", "saya", "ini"}

# Inisialisasi Tools
stemmer = StemmerFactory().create_stemmer()

In [95]:
# Definisi Pemegangan Frasa Kompleks
def substitute_complex_phrases(text, phrase_map):
    """Mengganti frasa kompleks dengan single token sebelum tokenisasi."""
    # Pastikan substitusi dilakukan pada teks lowercase
    text_lower = text.lower()
    
    for phrase, token in phrase_map.items():
        # Lakukan penggantian frasa dalam teks
        # Misalnya: "dataran tinggi" diganti menjadi "datarantinggi"
        text_lower = text_lower.replace(phrase, token)
        
    return text_lower

# PHRASE_MAP yang sudah didefinisikan
PHRASE_MAP = {
    # Geografis/Kondisi
    'dataran tinggi': 'datarantinggi',
    'dataran rendah': 'dataranrendah',
    'air terjun': 'airterjun',

    # Fasilitas
    'kamar mandi': 'kamarmandi',
    'tempat parkir': 'tempatparkir',
    'warung makan': 'warungmakan',
    'air bersih': 'airbersih',
    'listrik tersedia': 'listriktersedia',
    'akses jalan': 'aksesjalan',

    # Gabungkan istilah penting yang sering dicari
    'pemandangan indah': 'pemandanganindah',
    'suasana tenang': 'suasanatenang',
    'harga terjangkau': 'hargaterjangkau',
    'mudah diakses': 'mudahdiakses',
    'tempat kemah': 'tempatkemah',
}

In [96]:
# Definisi Pemetaan Region
REGION_MAP = {
    'jateng': 'jateng', 'jawa tengah': 'jateng', 'jawatengah': 'jateng',
    'diy': 'diy', 'jogja': 'diy', 'yogyakarta': 'diy', 'jogjakarta': 'diy'
}

def detect_region_and_filter_query(query_text):
    """
    Menganalisis query untuk menentukan apakah mengandung niat regional.
    Mengembalikan query yang sudah difilter (tanpa kata regional) dan kode region.
    """
    
    query_text_lower = query_text.lower()
    detected_region = None
    
    # Deteksi Region
    for term, region in REGION_MAP.items():
        if term in query_text_lower:
            detected_region = region
            query_text_lower = query_text_lower.replace(term, '') # Hapus kata regional
            # Break setelah region pertama terdeteksi (asumsi hanya 1 region per query)
            break 
            
    # Rebuild Query tanpa kata regional (untuk VSM)
    # Hapus spasi berlebihan dan filter token kosong
    filtered_query_text = " ".join([word for word in query_text_lower.split() if word])
    
    return filtered_query_text, detected_region

In [97]:
# Definisi Special Intent Map
SPECIAL_INTENT_MAP = {
    'semua tempat kemah': 'ALL', 
    'tempat kemah terbaik': 'RATING_TOP',
    'tempat kemah dengan ranking tertinggi': 'RATING_TOP',
    'tempat kemah terburuk': 'RATING_BOTTOM',
    
}

def detect_intent(query_text):
    """
    Menganalisis query untuk menentukan niat khusus (ALL/RATING).
    Mengembalikan query VSM yang sudah bersih, dan special_intent.
    """
    query_text_lower = query_text.lower()
    detected_region = None
    special_intent = None
    
    # 1. Deteksi Niat Khusus (ALL/RATING)
    for term, intent in SPECIAL_INTENT_MAP.items():
        if term in query_text_lower:
            special_intent = intent
            query_text_lower = query_text_lower.replace(term, '')
            break
            
    # 3. Rebuild Query VSM
    filtered_query_text = " ".join([word for word in query_text_lower.split() if word])
    
    return filtered_query_text, special_intent

In [98]:
# --- 2. DEFENISI FUNGSI HELPER & VSM CLASSES ---
# Fungsi Pembersihan Karakter Spesial
def remove_special_characters(text):
    if not isinstance(text, str):
        return "" 
    regex = re.compile(r'[^a-zA-Z0-9\s]')
    return re.sub(regex, '', text)

# Fungsi Proses Penuh (Preprocessing)
def full_preprocessing(text):
    if not isinstance(text, str):
        return []
        
    cleaned_text = remove_special_characters(text)
    cleaned_text = re.sub(r'\d', '', cleaned_text)

    text_with_phrases = substitute_complex_phrases(cleaned_text, PHRASE_MAP)
    
    # Simple Tokenization (split by whitespace) & Lowercasing
    words = text_with_phrases.lower().split()
    
    words = [w for w in words if w not in stopwords_id]
    
    # Stemming
    stemmed_words = [stemmer.stem(w) for w in words]
    
    final_words = [w for w in stemmed_words if len(w) > 1]
    return final_words

# Inverted Index Classes
class Node:
    def __init__(self, docId, freq=None):
        self.freq = freq # TF-IDF weight
        self.doc = docId
        self.nextval = None

class SlinkedList:
    def __init__(self, head=None):
        self.head = head

In [99]:
# --- 3. APLIKASI PREPROCESSING & HITUNG DF & IDF (INDEXING PHASE 1) ---
# Pastikan dataset sudah dimuat di df_corpus
df_corpus['Teks_Mentah'] = df_corpus['Teks_Mentah'].fillna('')
df_corpus['Clean_Tokens'] = df_corpus['Teks_Mentah'].apply(full_preprocessing)

N = len(df_corpus)
df_counts = {} # Document Frequency

for tokens in df_corpus['Clean_Tokens']:
    for word in set(tokens): 
        df_counts[word] = df_counts.get(word, 0) + 1

idf_scores = {}
for term, count in df_counts.items():
    idf_scores[term] = math.log10(N / count)

In [100]:
# --- 4. BUILDING THE INVERTED INDEX WITH TF-IDF (INDEXING PHASE 2) ---
linked_list_data = {}
unique_words_all = set(df_counts.keys())

for word in unique_words_all:
    linked_list_data[word] = SlinkedList()
    linked_list_data[word].head = Node(docId=0, freq=None) 

for index, row in df_corpus.iterrows():
    doc_id = row['Doc_ID']
    tokens = row['Clean_Tokens']
    
    tf_in_doc = {}
    for word in tokens:
        tf_in_doc[word] = tf_in_doc.get(word, 0) + 1

    for term, tf in tf_in_doc.items():
        tfidf = tf * idf_scores[term]
        
        linked_list = linked_list_data[term].head
        while linked_list.nextval is not None:
            linked_list = linked_list.nextval
        
        linked_list.nextval = Node(docId=doc_id, freq=tfidf)

# Mapping Doc ID to Name and Rating for final result
df_metadata = df_corpus[['Doc_ID', 'Nama_Tempat', 'Lokasi', 'Rating']].copy()
avg_rating_per_place = df_metadata.groupby('Nama_Tempat')['Rating'].mean().reset_index()
avg_rating_per_place.rename(columns={'Rating': 'Avg_Rating'}, inplace=True)
df_metadata = df_metadata.merge(avg_rating_per_place, on='Nama_Tempat', how='left')
df_metadata.set_index('Doc_ID', inplace=True)

In [101]:
# --- 5. FUNGSI VSM RANKING MURNI ---
def search_by_keyword(query_text, region=None):
    """
    Melakukan pencarian VSM murni berdasarkan kata kunci (relevansi ulasan).
    """
    
    # Panggil deteksi region dan filter query
    vsm_query_text, region_filter = detect_region_and_filter_query(query_text)
    
    # Jika query menjadi kosong setelah filter regional, gunakan kata kunci default
    if not vsm_query_text or vsm_query_text.isspace():
        vsm_query_text = "camping" # Gunakan kata kunci generik 'camping' jika query kosong
    
    # 1. Preprocessing Query
    query_tokens = full_preprocessing(query_text)
    
    if not query_tokens:
        return []
    
    # 2. Query Vectorization (TF-IDF)
    query_tf = {}
    for word in query_tokens:
        query_tf[word] = query_tf.get(word, 0) + 1
        
    query_weights = {}
    involved_docs = set()
    
    for term, tf in query_tf.items():
        if term in idf_scores:
            query_weights[term] = tf * idf_scores[term]
            
            # Collect all documents involved from the Index
            current_node = linked_list_data[term].head.nextval
            while current_node is not None:
                involved_docs.add(current_node.doc)
                current_node = current_node.nextval
        else:
            continue

    if not involved_docs:
        return []

    # 3. Cosine Similarity (Dot Product Only)
    doc_scores = {doc_id: 0 for doc_id in involved_docs}
    
    # Calculate DOT PRODUCT: Sum(W(t,d) * W(t,q))
    for term, W_q in query_weights.items():
        current_node = linked_list_data[term].head.nextval
        while current_node is not None:
            doc_id = current_node.doc
            W_d = current_node.freq # TF-IDF weight W(t,d)
            doc_scores[doc_id] += W_d * W_q
            current_node = current_node.nextval
            
    # 4. Ranking Ulasan (Doc ID)
    ranked_results_by_doc = sorted(doc_scores.items(), key=lambda item: item[1], reverse=True)
    
    # 5. Agregasi ke Nama Tempat (Mengambil ulasan paling relevan per tempat)
    final_recommendations = []
    unique_names = set()
    
    for doc_id, vsm_score in ranked_results_by_doc:
        meta = df_metadata.loc[doc_id]
        
        # Filter berdasarkan region jika diminta (opsional)
        if region_filter:
            # Contoh: jika region_filter='diy', Lokasi harus mengandung 'DIY'/'Jogja'
            # Kita bandingkan region filter dengan string Lokasi
            if region_filter not in meta['Lokasi'].lower():
                continue # Skip dokumen yang tidak sesuai region
            
        name = meta['Nama_Tempat']
        
        if name not in unique_names:
            unique_names.add(name)
            final_recommendations.append({
                'name': name,
                'location': meta['Lokasi'],
                'avg_rating': meta['Avg_Rating'],
                'top_vsm_score': vsm_score, # Skor VSM dari ulasan paling relevan
            })
            
    return final_recommendations

In [102]:
print("==================================================")
print("MESIN PENCARIAN REKOMENDASI TEMPAT KEMAH VSM SIAP!")
print("==================================================")
print("Anda dapat memasukkan kata kunci untuk mencari rekomendasi.")

while True:
    # Mengambil input query dari pengguna
    query_text = input("\nMasukkan kata kunci pencarian (atau ketik 'keluar' untuk berhenti): \n").strip()
    
    if query_text.lower() in ('keluar', 'exit', 'berhenti', 'quit', 'stop', 'kembali'):
        print("\nSesi pencarian diakhiri. Terima kasih!")
        break
    
    if not query_text:
        continue
        
    # Panggil fungsi pencarian VSM
    vsm_ranking = search_by_keyword(query_text, region=None) 

    # Deteksi region untuk ditampilkan
    _, detected_region = detect_region_and_filter_query(query_text)
    filter_status = f"Filtered by: {detected_region.upper()}" if detected_region else "No Region Filter Applied"
    
    print("\n--------------------------------------------------")
    print(f"HASIL PENCARIAN untuk: '{query_text}'")
    print(f"Kata Kunci Diproses: {full_preprocessing(query_text)}")
    print("--------------------------------------------------")

    if vsm_ranking:
        print("Rekomendasi Tempat Kemah (Diurutkan berdasarkan Relevansi Ulasan Tertinggi):")
        
        for i, item in enumerate(vsm_ranking):
            print(f"{i+1}. {item['name']}")
            print(f"   | Lokasi: {item['location']}")
            print(f"   | Rata-rata Rating Tempat: {item['avg_rating']:.2f}")
            print(f"   | Skor Relevansi (VSM Score): {item['top_vsm_score']:.4f}")
        
        # Logika lanjut
        continue_input = input("\nApakah Anda ingin melanjutkan pencarian? (ya/tidak): ").strip().lower()
        
        if continue_input not in ('ya', 'y'):
            print("\nSesi pencarian diakhiri. Terima kasih!")
            break
            
        # Hapus output sebelum loop selanjutnya
        clear_output(wait=True) 
        print("Mesin pencarian siap untuk query selanjutnya...")
    else:
        print("Tidak ditemukan tempat kemah yang relevan dengan kata kunci ini.")
        
        # Logika lanjut
        continue_input = input("\nApakah Anda ingin melanjutkan pencarian? (ya/tidak): ").strip().lower()
        
        if continue_input not in ('ya', 'y'):
            print("\nSesi pencarian diakhiri. Terima kasih!")
            break
            
        # Hapus output sebelum loop selanjutnya
        clear_output(wait=True) 
        print("Mesin pencarian siap untuk query selanjutnya...")

Mesin pencarian siap untuk query selanjutnya...

--------------------------------------------------
HASIL PENCARIAN untuk: 'tempat kemah terbaik di jawa tengah'
Kata Kunci Diproses: ['tempatkemah', 'baik', 'jawa']
--------------------------------------------------
Rekomendasi Tempat Kemah (Diurutkan berdasarkan Relevansi Ulasan Tertinggi):
1. Ratan Lurung Basecamp Gedongsongo
   | Lokasi: Kab. Semarang, Jateng
   | Rata-rata Rating Tempat: 4.80
   | Skor Relevansi (VSM Score): 11.2521
2. Camping Umbul Sidomukti
   | Lokasi: Kab. Semarang, Jateng
   | Rata-rata Rating Tempat: 4.60
   | Skor Relevansi (VSM Score): 3.3140
3. Camp Ground Bukit Sikunir
   | Lokasi: Wonosobo, Jateng
   | Rata-rata Rating Tempat: 4.60
   | Skor Relevansi (VSM Score): 1.6570
4. Pinusan Nglimut
   | Lokasi: Kendal, Jateng
   | Rata-rata Rating Tempat: 4.20
   | Skor Relevansi (VSM Score): 1.6570
5. Camp ground puncak ungaran
   | Lokasi: Kab. Semarang, Jateng
   | Rata-rata Rating Tempat: 4.50
   | Skor Relevan